In [ ]:
import sys
sys.path.append('..')
from core.utilities.notebook_setups.notebook_display import *
set_up(log_lvl=LogLvl.DEBUG)

# Objects

In [ ]:
import sys
try:
    del sys.modules["data_graphing"] 
    del data_graphing
except:
    pass

from data_graphing import ColorHandler, PlotterHelper, prettify_frame

In [ ]:
class FramedDataFuncs:   
    def __init__(self, DATA):
        self._DATA = DATA
        self._PLOTTER = PlotterHelper(DATA)
        self._ARCH_FUNCS = ArchFuncs(self._DATA)
        self._SNGL_CARD_FUNCS = SingleCardFuncs(self._DATA)
    
    @property
    def SET(self):
        """The draft set."""
        return self._DATA.SET
    
    @property
    def FULL_SET(self):
        """The full name of the draft set."""
        return self._DATA.FULL_SET
    
    @property
    def FORMAT(self):
        """The format type."""
        return self._DATA.FORMAT
    
    @property
    def SHORT_FORMAT(self):
        """The shorthand of the format type."""
        return self._DATA.FULL_FORMAT
    
    @property
    def DATA(self):
        """The object which contains the data about the set and format."""
        return self._DATA
    
    @property
    def PLOTTER(self):
        """The object which helps to plot and save data as pngs and the like."""
        return self._PLOTTER
        
    def check_for_updates(self):
        """Populates and updates all data properties, filling in missing data."""
        self._DATA.check_for_updates()

    def reload_data(self):
        """Populates and updates all data properties, reloading all data."""
        self._DATA.reload_data()  
    
    def deck_group_frame(self, name=None, date=None, summary=False):
        """Returns a subset of the 'GROUPED_ARCHETYPE' data as a DataFrame."""
        return self.DATA.deck_group_frame(name, date, summary)
     
    def deck_archetype_frame(self, deck_color=None, date=None, summary=False):
        """Returns a subset of the 'SINGLE_ARCHETYPE' data as a DataFrame."""
        return self.DATA.deck_archetype_frame(deck_color, date, summary)
    
    def card_frame(self, name=None, deck_color=None, date=None, summary=False):
        """Returns a subset of the 'CARD' data as a DataFrame."""
        return self.DATA.card_frame(name, deck_color, date, summary)
    
    def compress_date_range_data(self, start_date, end_date, card_name=None):
        """Summarizes card data over a provided set of time."""
        return self.DATA.compress_date_range_data(start_date, end_date, card_name)
        
    
    #ArchFuncs
    def get_games_played(self, deck_color):
        return self._ARCH_FUNCS.get_games_played(deck_color)
        
    def get_avg_winrate(self, day=None, arch='All Decks'):
        return self._ARCH_FUNCS.get_avg_winrate(day, arch)
    
    def get_archetype_frame(self, colors, roll=None):
        return self._ARCH_FUNCS.get_archetype_frame(colors, roll)

    def get_archetype_winrate_history(self, color_filter=None, roll=None, *, save=False):
        return self._ARCH_FUNCS.get_archetype_winrate_history(color_filter, roll, save=save)

    def plot_archetype_winrate_history(self, color_filter=None, roll=None, derivs=0, color_dict=None, pref=''):
        return self._ARCH_FUNCS.plot_archetype_winrate_history(color_filter, roll, derivs, color_dict, pref)       
        
    def get_archetype_playrate_history(self, color_filter=None, color_count=0, roll=None, *, save=False):
        return self._ARCH_FUNCS.get_archetype_playrate_history(color_filter, color_count, roll, save=save)

    def plot_archetype_playrate_history(self, color_filter=None, color_count=0, roll=None, derivs=0, color_dict=None, pref=''):
        return self._ARCH_FUNCS.plot_archetype_playrate_history(color_filter, color_count, roll, derivs, color_dict, pref)
    
    def card_relative_winrates(self, deck_colors, win_rate_col, filter_option=None):
        return self._ARCH_FUNCS.card_relative_winrates(deck_colors, win_rate_col, filter_option)
    
    #SingleCardFuncs
    def get_card_summary(self, card_name, colors='', roll=None):
        return self._SNGL_CARD_FUNCS.get_card_summary(card_name, colors, roll)

    def plot_card_summary(self, card_name, colors='', roll=None):
        return self._SNGL_CARD_FUNCS.plot_card_summary(card_name, colors, roll)
    
    def get_pick_stats(self, card_name, roll=None):
        return self._SNGL_CARD_FUNCS.get_pick_stats(card_name, roll)
    
    def plot_pick_stats(self, card_name, roll=None):
        return self._SNGL_CARD_FUNCS.plot_pick_stats(card_name, roll)
    
    def card_archetype_performance(self, card_name, color_cols=None):
        return self._SNGL_CARD_FUNCS.card_archetype_performance(card_name, color_cols)
    
    def stat_archetype_performance(self, stat_name, color_cols=None):
        return self._SNGL_CARD_FUNCS.stat_archetype_performance(stat_name, color_cols)
    
    def compare_card_evaluations(self, start_date=None, end_date=None):
        def inner_func(date):
            df = self.card_frame(date=date, deck_color='')
            df.index = [tup[2] for tup in df.index]
            return df
        
        if start_date is None:
            metadata = SetMetadata.get_metadata(self.SET)
            start_date = metadata.RELEASE_DATE
            
        if end_date is None:
            end_date = date.today() - timedelta(days=1)

        first = inner_func(date=start_date)
        last = inner_func(date=end_date)
        diff = last[['ALSA', 'ATA', 'Color', 'Rarity']].copy()
        diff['ALSA Change'] = first['ALSA'] - last['ALSA']
        diff['ATA Change'] = first['ATA'] - last['ATA']
        return diff[['ALSA', 'ALSA Change', 'ATA', 'ATA Change', 'Color', 'Rarity']]


    def get_top(self, column, count=10, asc=True, card_color=None, card_rarity=None, deck_color='', play_lim=None):
        frame = self.card_frame(deck_color=deck_color, summary=True, card_rarity=card_rarity)
        frame = frame.sort_values(column, ascending=asc)

        if card_color is not None:
            card_color = WUBRG.get_color_identity(card_color)
            frame = frame[frame['Color'] == card_color]

        if play_lim is not None:
            #TODO: Fix this. 
            if type(play_lim) is float: play_lim *= self.get_games_played(deck_color)
            print(f'Minimum Games played to be included: {play_lim}')
            frame = frame[frame['# GP'] >= play_lim]

        return frame.head(count)

In [ ]:
from data_graphing import ROLL, ARCHETYPES_COLOR_DICT
from wubrg import COLOR_PAIRS, COLOUR_GROUPINGS
from data_fetching.utils.consts import COLOR_COUNT_REVERSE_MAP, COLOR_COUNT_SHORTHAND, COLOR_COUNT_SHORTHAND_MAP

class ArchFuncs:   
    def __init__(self, DATA):
        self._DATA = DATA
        
    @staticmethod
    def _get_play_stat_frame(frame, col, roll=None, aggfunc=None):
        if roll is None: roll = ROLL
        if aggfunc is None: aggfunc = np.sum
        ret = frame[[col]]
        ret.reset_index(inplace=True, level=1)
        ret = ret.pivot_table(values=col, index=ret.index, columns='Name', dropna=False, aggfunc=aggfunc)
        ret = ret.rolling(window=roll, min_periods=1, center=True).mean()
        ret.columns.names = [col]
        return ret

    def _get_all_stat(self, stat, roll=None, aggfunc=None):
        archetypes = self._get_play_stat_frame(self._DATA.deck_archetype_frame(), stat, roll, aggfunc)
        archetypes = archetypes[[color for color in COLOR_COMBINATIONS if color in archetypes.columns]]

        groups = self._get_play_stat_frame(self._DATA.deck_group_frame(), stat, roll, aggfunc)
        groups.rename(columns=COLOR_COUNT_SHORTHAND_MAP, inplace=True)
        groups = groups[COLOR_COUNT_SHORTHAND]

        return pd.concat([groups, archetypes], axis=1)


    def get_winrates(self, roll=None):
        wins = self._get_all_stat('Wins', roll, np.sum)
        games = self._get_all_stat('Games', roll, np.sum)
        frame = (wins / games) * 100
        frame.columns.names = ['Avg. Win %']
        return frame


    def get_playrates(self, num_colors=0, roll=None):
        #TODO: Remove num_colors, and handle bubbling chnages from that.
        games = self._get_all_stat('Games', roll, np.sum)
        
        if num_colors == 0:
            frame = games.div(games['ALL'], axis='rows') * 100
            frame.columns.names = ['% of Decks']
            return frame

        true_name = COLOR_COUNT_REVERSE_MAP[num_colors].title()
        summ_id = COLOR_COUNT_SHORTHAND_MAP[true_name]

        frame = games.div(games[summ_id], axis='rows') * 100
        frame.columns.names = [f'% of {num_colors}C Decks']
        return frame
   
    def get_archetype_winrate_history(self, color_filter=None, roll=None, *, save=False):
        if roll is None: roll = ROLL
            
        frame = self.get_winrates(roll)
       
        if isinstance(color_filter, str):
            # TODO: Make this more flexible.
            #col_filt = ['ALL', f'{len(color_filter)}C'] + [col for col in COLOR_PAIRS if color_filter in col]  
            col_filt = ['ALL', '2C'] + [col for col in COLOR_PAIRS if color_filter in col]  
            frame = frame[col_filt]
        elif isinstance(color_filter, list):
            frame = frame[color_filter]
            
        if save:
            PlotterHelper(self._DATA).frame_to_png(frame, "archetype_winrate_table.png")
        
        return frame        
        
    #TODO: Implement a more generic version of this that takes in a list of deck colours to include as output. 
    def get_archetype_playrate_history(self, color_filter=None, color_count=0, roll=None, *, save=False):
        if roll is None: roll = ROLL
            
        frame = self.get_playrates(color_count, roll)

        if isinstance(color_filter, str):
            col_filt = [col for col in COLOR_PAIRS if color_filter in col]
            frame = frame[col_filt]
        if isinstance(color_filter, list):
            frame = frame[color_filter]
            
        if save:
            PlotterHelper(self._DATA).frame_to_png(frame, "archetype_playrate_table.png")

        return frame
    
    def plot_archetype_winrate_history(self, color_filter=None, roll=None, derivs=0, color_dict=None, pref=''):
        if roll is None: roll = ROLL
        if color_dict is None: color_dict = ARCHETYPES_COLOR_DICT

        data = self.get_archetype_winrate_history(color_filter, roll)
        for _ in range(0, derivs):
            data = data.diff()
        data.index = [idx[5:] for idx in data.index]
        colors = str(color_filter)  #TODO: Make this cleverer at showing filter.

        plot_help = PlotterHelper(self._DATA, color_dict=color_dict)
        fig, ax = plot_help.new_single_plot('Archetype Winrates', width=16, height=8)
        plot_help.accredit(y=0.035, x=0.51)
        plot_help.desc_note(colors=colors, roll=roll, y=0.935, x=0.51)
        
        plot_help.set_labels(x_label="Date", y_label="Win Rate")
        plot_help.set_data(data, color_filter)
        
        if not pref:
            plot_help.save_fig(f"win_rates_{roll}day_avg.png", "Metagame")
        else:
            plot_help.save_fig(f"{pref}_win_rates_{roll}day_avg.png", "Metagame")

    def plot_archetype_playrate_history(self, color_filter=None, color_count=0, roll=None, derivs=0, color_dict=None, pref=''):
        if roll is None: roll = ROLL
        if color_dict is None: color_dict = ARCHETYPES_COLOR_DICT
            
        data = self.get_archetype_playrate_history(color_filter, color_count, roll)
        for _ in range(0, derivs):
            data = data.diff()
        data.index = [idx[5:] for idx in data.index]
        
        
        plot_help = PlotterHelper(self._DATA, color_dict=color_dict)
        fig, ax = plot_help.new_single_plot('Archetype Playrates', width=16, height=8)
        plot_help.accredit(y=0.035, x=0.51)
        plot_help.desc_note(colors=color_filter, roll=roll, y=0.935, x=0.51)
        
        plot_help.set_labels(x_label="Date", y_label="Percent of Metagame")
        plot_help.set_data(data, color_filter)
        
        if not pref:
            plot_help.save_fig(f"play_rates_{roll}day_avg.png", "Metagame")
        else:
            plot_help.save_fig(f"{pref}_play_rates_{roll}day_avg.png", "Metagame")
            
    
    def card_relative_winrates(self, deck_colors=None, win_rate_col='GIH WR', filter_option=None):
        deck_colors = get_color_identity(deck_colors)
        
        # Get the relevant list of cards, and then trim it down to the relevant colours.
        sub_frame = self._DATA.card_frame(deck_color=deck_colors)
        sub_frame = set_data.BO1.DATA.CARD_HISTORY_FRAME.loc[slice(None), get_color_slice(deck_colors), get_name_slice(None)]
        sub_frame = sub_frame.reset_index(level=1)
        if deck_colors:
            sub_frame = sub_frame[sub_frame['Cast Color'].isin(WUBRG.get_color_subsets(deck_colors))]

        # Get the winrates for the cards and average winrate of the archetype, then re-center the card winrates with it. 
        win_frame = self._get_play_stat_frame(sub_frame, win_rate_col, roll=1, aggfunc=np.mean)
        avg_frame = self.get_archetype_winrate_history(deck_colors)
        target_avg = deck_colors if deck_colors else 'ALL'
        ret_frame = win_frame.sub(avg_frame[target_avg], axis='rows').T

        # Get the games played from the subset of cards and use it to re-weight the win rates per day, to calculate an accurate average.
        games_frame = self._get_play_stat_frame(sub_frame, '# GP', roll=1, aggfunc=np.mean).T
        ret_frame[f'AVG'] = (ret_frame * games_frame).sum(axis=1) / games_frame.sum(axis=1)
        ret_frame['# GP'] = games_frame.sum(axis=1)

        # Sort by most winning first.
        ret_frame = ret_frame.sort_values(f'AVG', ascending=False).T
        return ret_frame[::-1]

In [ ]:
from data_graphing import ROLL, STATS_COLOR_DICT
from wubrg import COLOR_COMBINATIONS
from wubrg.consts import COLOR_PAIRS


class SingleCardFuncs:   
    def __init__(self, DATA):
        self._DATA = DATA
        self._COLOR_IDX = 0
        
    def _shorten_data(self, card_name, roll, cols, colors=''):
        frame = self._DATA.card_frame(name=card_name, deck_color=colors)[cols]
        frame.index = [tup[0][5:] for tup in frame.index]
        rolling = frame.rolling(window=roll, min_periods=1, center=True).mean()
        return rolling

    def plot_card_summary(self, card_name, colors='', roll=None, color_dict=None):
        if roll is None: roll = ROLL
        if color_dict is None: color_dict = STATS_COLOR_DICT
        rolling = self._shorten_data(card_name, roll, ['GIH WR', 'GND WR', 'ATA', 'ALSA', '# GP', '# GIH', '# Picked', '# Seen'], colors=colors)
        if rolling.empty:
            print(f"""Could not find data for "{card_name}". Please make sure it is spelled correctly, or you are accessing the right set.""")
            return False
        
        plot_help = PlotterHelper(self._DATA, color_dict=color_dict)
        fig, ax = plot_help.new_quad_plot(card_name)
        plot_help.accredit(y=0.075)
        plot_help.desc_note(colors=colors, roll=roll)
                
        plot_help.set_labels(y_label="Win Percent", g_x=0, g_y=0)
        plot_help.set_data(rolling, ['GIH WR', 'GND WR'], g_x=0, g_y=0)
        
        plot_help.set_labels(y_label="Pick Number", g_x=0, g_y=1)
        plot_help.set_data(rolling, ['ALSA', 'ATA'], inv_y=True, g_x=0, g_y=1)
        
        plot_help.set_labels(x_label="Date", y_label="# of Games", g_x=1, g_y=0)
        plot_help.set_data(rolling, ['# GP', '# GIH'], g_x=1, g_y=0)

        plot_help.set_labels(x_label="Date", y_label="# of Cards", g_x=1, g_y=1)
        plot_help.set_data(rolling, ['# Seen', '# Picked'], g_x=1, g_y=1)
        
        if colors:
            plot_help.save_fig(f"pcs_{card_name}_{colors}.png", "Summary")
        else:
            plot_help.save_fig(f"pcs_{card_name}.png", "Summary")
        
        return True
    
    
    def plot_pick_stats(self, card_name, roll=None, color_dict=None):
        if roll is None: roll = ROLL        
        if color_dict is None: color_dict = STATS_COLOR_DICT
        taken_data = self._shorten_data(card_name, roll, ['ALSA', 'ATA'])
        if taken_data.empty:
            print(f"""Could not find data for "{card_name}". Please make sure it is spelled correctly, or you are accessing the right set.""")
            return
        
        plot_help = PlotterHelper(self._DATA, color_dict=color_dict)
        fig, ax = plot_help.new_single_plot(card_name)
        plot_help.accredit()
        plot_help.desc_note(roll=roll, y=0.96)
        
        plot_help.set_labels(x_label="Date", y_label="Pick Number")
        plot_help.set_data(taken_data, ['ALSA', 'ATA'], inv_y=True)
        
        plot_help.save_fig(f"pps_{card_name}.png", "Pick Stats")


    def card_archetype_performance(self, card_name, color_cols=None):
        frame = self._DATA.card_frame(card_name, summary=True).T
        frame.loc[STAT_COL_NAMES].T
        if color_cols is not None:
            ret = ret[color_cols]
        return frame
    
    
    def stat_archetype_performance(self, stat_name, color_cols=None):
        series = self._DATA.card_frame(summary=True)[stat_name]
        frame = series.reset_index(level=0)
        ret = pd.pivot_table(frame, index='Name', columns='Deck Colors')
        ret.columns = ret.columns.droplevel(0)
        if color_cols is not None:
            ret = ret[color_cols]
        return ret

# Initialization

# Data Overview

In [ ]:
examiner = FramedDataFuncs(set_data.BO1)
examiner.card_frame(deck_color='', summary=True)

In [ ]:
raise Exception('Stopping Auto-Run!')

# TODO

- Calculate archetype openess
 - GIH WR & ALSA based
 - 2.25 of a common per draft 
- Improve graphing capabilities
 - Modify Graphs so they also save an image to a cache folder.
 - Implement better graph titles and axes
 - Update functions to be more general, and have DataFrames piped into them.
- Move sets of files into subfolders based on functionality and relationship (DataFetch, Cards, Graphing, Utils, etc.)
- Better group settings and consts into one file/location.

## Current Tests

In [ ]:
set_data.BO1.card_frame("Gaea's Might", summary=True)[['# GP', 'GIH WR']]

TODO: Update code so that
 1) Cards without data aren't treated as having a 0% win rate.
 2) Ensure that NaNs are respected so the average improvement is calculated properly.

In [ ]:
sub_frame = set_data.BO1.DATA.CARD_HISTORY_FRAME.loc[get_date_slice(('2022-04-28', '2022-05-01')), get_color_slice('WR'), get_name_slice(None)]
sub_frame.reset_index(level=1, inplace=True)
sub_frame = sub_frame[sub_frame['Cast Color'].isin(WUBRG.get_color_subsets('WR'))]
sub_frame

In [ ]:
sub_frame.loc[(slice(None), slice('Depopulate', 'Depopulate', None)), :]

In [ ]:
temp_frame = sub_frame.copy()
temp_frame.reset_index(inplace=True, level=1)
#temp_frame

In [ ]:
temp_frame.pivot_table(values='GIH WR', index=temp_frame.index, columns='Name', dropna=False)

In [ ]:
# Get the winrates and average winrate of the colour group, then re-center the card winrates with it. 
win_frame = ArchFuncs._get_play_stat_frame(sub_frame, 'GIH WR', roll=1)
win_frame

In [ ]:
TRG_DECK_COLOR = 'WR'
TRG_WIN_RATE = 'GIH WR'

def card_relative_winrates(deck_colors, win_rate_col, filter_option=None):
    # Get the relevant list of cards, and then trim it down to the relevant colours.
    sub_frame = set_data.BO1.DATA.CARD_HISTORY_FRAME.loc[slice(None), get_color_slice(deck_colors), get_name_slice(None)]
    sub_frame = sub_frame.reset_index(level=1)
    sub_frame = sub_frame[sub_frame['Cast Color'].isin(WUBRG.get_color_subsets(deck_colors))]

    # Get the winrates and average winrate of the colour group, then re-center the card winrates with it. 
    win_frame = ArchFuncs._get_play_stat_frame(sub_frame, win_rate_col, roll=1, aggfunc=np.mean)
    avg_frame = examiner.get_archetype_winrate_history([deck_colors])
    centred_frame = win_frame.sub(avg_frame[deck_colors], axis='rows').T

    # Get the games played from the subset of cards and use it to re-weight the win rates per day, to calculate an accurate average.
    games_frame = ArchFuncs._get_play_stat_frame(sub_frame, '# GP', roll=1, aggfunc=np.mean).T
    ret_frame = centred_frame
    ret_frame['AVG'] = (ret_frame * games_frame).sum(axis=1) / games_frame.sum(axis=1)
    ret_frame['GAMES'] = games_frame.sum(axis=1)

    # Sort by most winning first.
    ret_frame = ret_frame.sort_values('AVG', ascending=False).T
    return sub_frame, win_frame, games_frame, avg_frame, ret_frame, centred_frame
    
frames = card_relative_winrates(TRG_DECK_COLOR, TRG_WIN_RATE)
frames[4]

In [ ]:
def gt(x, y):  #Used for function pointer shenanigans
    return x > y

def lt(x, y):  #Used for function pointer shenanigans
    return x < y

def filter_quadrant_cards_df(func1, func2, iwd_thresh=0, play_lim=0.01, card_rarity=None, deck_color=''):
    frame = set_data.BO1.card_frame(deck_color=deck_color, summary=True)
    if card_rarity is not None:
        frame = frame[frame['Rarity'] == card_rarity]
    
    # TODO: Make the mean different based on card rarity so cards aren't moved into incorrect categories.
    
    if play_lim is not None:
        if type(play_lim) is float: play_lim *= set_data.BO1.get_games_played(deck_color)
        print(f'Minimum Games played to be included: {play_lim}')
        frame = frame[frame['# GP'] >= play_lim]
        
    games_played_mean = frame['# GP'].mean()
    cards = frame[func1(frame['IWD'], iwd_thresh)]
    cards = cards[func2(cards['# GP'], games_played_mean)]
    cards = cards.sort_values('IWD', ascending=func1==lt)
    return cards


def get_trap_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_quadrant_cards_df(lt, gt, iwd_thresh, play_lim, card_rarity, deck_color)

def get_niche_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_quadrant_cards_df(gt, lt, iwd_thresh, play_lim, card_rarity, deck_color)

def get_staple_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_quadrant_cards_df(gt, gt, iwd_thresh, play_lim, card_rarity, deck_color)

def get_dreg_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_quadrant_cards_df(lt, lt, iwd_thresh, play_lim, card_rarity, deck_color)

In [ ]:
get_niche_cards(deck_color='GW', card_rarity='CU')

In [ ]:
set_data.BO1.DATA.SINGLE_ARCHETYPE_SUMMARY_FRAME.loc[['W']]

In [ ]:
from data_fetching.utils.index_slice_helper import get_name_slice, get_color_slice

In [ ]:
set_data.BO1.DATA.CARD_HISTORY_FRAME.loc[slice('2022-04-29', '2022-05-01'), get_color_slice(''), get_name_slice(['Girder Goons', 'Angelic Observer', 'A Little Chat', 'Case the Joint'])]

In [ ]:
data = set_data.BO1.DATA.CARD_HISTORY_FRAME.loc(axis=0)[pd.IndexSlice[slice(None), get_color_slice(''), get_name_slice(None)]]
cheap = data[data['CMC'] <= 3]
expensive = data[data['CMC'] >= 5]

In [ ]:
cheap['Games Won'] = cheap['# GIH'] * cheap['GIH WR'] * 0.01
cheap_sums = cheap[['# GP', '# GIH', 'Games Won']].sum()
cheap_wr = cheap_sums['Games Won'] / cheap_sums['# GIH']
cheap_wr

In [ ]:
expensive['Games Won'] = expensive['# GIH'] * expensive['GIH WR'] * 0.01
expensive_sums = expensive[['# GP', '# GIH', 'Games Won']].sum()
expensive_wr = expensive_sums['Games Won'] / expensive_sums['# GIH']
expensive_wr

In [ ]:
for i in range(0, 9):
    by_cmc = data[data['CMC'] == i]
    by_cmc['Games Won'] = by_cmc['# GIH'] * by_cmc['GIH WR'] * 0.01
    sums = by_cmc[['# GP', '# GIH', 'Games Won']].sum()
    avg = by_cmc[['GIH WR']].mean()['GIH WR']
    wr = sums['Games Won'] / sums['# GIH'] * 100
    print(f"CMC: {i}, WR: {wr}  AVG: {avg}")

In [ ]:
examiner.plot_archetype_playrate_history(['2C', '3C', 'WU', 'WG', 'WUG'], roll=3, derivs=2)

In [ ]:
examiner.plot_archetype_playrate_history(['2C', '3C', 'WU', 'WG', 'UB', 'BR', 'RG', 'WUB', 'WUG', 'UBR', 'BRG', 'WRG'], roll=1, derivs=1)

In [ ]:
get_grade_distribution('')

In [ ]:
frame = examiner.card_frame(deck_color='', summary=True)

In [ ]:
mu, std = norm.fit(frame['GIH WR'])

In [ ]:
p = norm.cdf(frame['GIH WR'], mu, std) * 100
p

In [ ]:
frame_copy = frame.copy()
frame_copy['STD'] = p
frame_copy[['GIH WR', 'STD']].sort_values("GIH WR")

# Deck Comparisons

In [ ]:
from IPython.core.display import display, HTML
import sys
sys.path.append('..')
display(HTML("<style>.container { width:95% !important; }</style>"))
print(sys.version)

In [ ]:
from core.game_metadata.game_objects.Card import CardManager
CardManager.load_cache_from_file()

In [ ]:
from core.game_metadata import Deck, LimitedDeck, ConstructedDeck
from core.data_requesting.Request17Lands import Request17Lands
from core.wubrg.sorting import WUBRG_COLOR_INDEXES
from collections import OrderedDict
from core.game_metadata.utils.consts import RARITY_INDEXES
import re

In [ ]:
def compare_decks(d1, d2):
    def sort_key(x): return (x.CMC, WUBRG_COLOR_INDEXES[x.CAST_IDENTITY], x.NAME)
    
    diff = (d1 - d2)[0]
    over = (d1 | d2)[0]
    diff_1 = {k: diff[k] for k in diff if diff[k] > 0}
    diff_2 = {k: -diff[k] for k in diff if diff[k] < 0}
    
    over = {key:over[key] for key in sorted(over.keys(), key=sort_key)}
    diff_1 = {key:diff_1[key] for key in sorted(diff_1.keys(), key=sort_key)}
    diff_2 = {key:diff_2[key] for key in sorted(diff_2.keys(), key=sort_key)}
    return diff_1, over, diff_2

def format_diffs(diff_1, over, diff_2):
    diff_1_str = [f"{v:2} {k.NAME:30}" for k, v in diff_1.items()]
    over_str = [f"{v:2} {k.NAME:30}" for k, v in over.items()]
    diff_2_str = [f"{v:2} {k.NAME:30}" for k, v in diff_2.items()]
    
    x = max(len(diff_1_str), len(over_str), len(diff_2_str))

    def safe_val(l, i):
        try:
            return l[i]
        except IndexError:
            return ' ' * 33

    ret = '         Deck 1                           Shared                           Deck 2                  ' + '\n'
    for i in range(0, x):
        ret += safe_val(diff_1_str, i) + safe_val(over_str, i) + safe_val(diff_2_str, i) + '\n'
    return ret

def print_deck_comparison(d1, d2):
    diff_1, over, diff_2 = compare_decks(d1, d2)
    diff_str = format_diffs(diff_1, over, diff_2)
    print(diff_str)

In [ ]:
thiers = """Deck
1 Drowned Catacomb (XLN) 253
1 Shipwreck Marsh (MID) 267
1 Deathcap Glade (VOW) 261
1 Woodland Cemetery (DOM) 248
1 Dreamroot Cascade (VOW) 262
1 Hinterland Harbor (DOM) 240
1 Indatha Triome (IKO) 248
1 Zagoth Triome (IKO) 259
1 Fabled Passage (ELD) 244
5 Forest (MH2) 489
3 Island (BRO) 270
1 Essence Scatter (M19) 54
1 Heartless Act (IKO) 91
1 Courier's Briefcase (SNC) 142
1 Gala Greeters (SNC) 148
1 Inscription of Abundance (ZNR) 186
1 Paradise Druid (WAR) 171
1 Ranger Class (AFR) 202
1 Scavenging Ooze (M21) 204
1 Coldsteel Heart (CSP) 136
1 Aether Channeler (DMU) 42
1 Chatterfang, Squirrel General (MH2) 151
1 Chitterspitter (MH2) 153
1 Rishkar, Peema Renegade (JMP) 425
1 Tireless Tracker (SOI) 233
1 Ulvenwald Oddity (VOW) 225
1 Ertai Resurrected (DMU) 199
1 Binding the Old Gods (KHM) 206
1 Verdurous Gearhulk (KLR) 185
1 Enter the God-Eternals (WAR) 196
1 Shark Typhoon (IKO) 67
1 Titan of Industry (SNC) 159
1 Koma, Cosmos Serpent (KHM) 221
1 Cityscape Leveler (BRO) 233
"""

In [ ]:
mine = """Deck
1 Drowned Catacomb (XLN) 253
1 Shipwreck Marsh (MID) 267
1 Deathcap Glade (VOW) 261
1 Woodland Cemetery (DOM) 248
1 Dreamroot Cascade (VOW) 262
1 Hinterland Harbor (DOM) 240
1 Ketria Triome (IKO) 250
1 Indatha Triome (IKO) 248
1 Zagoth Triome (IKO) 259
1 Fabled Passage (ELD) 244
3 Forest (ONE) 276
3 Island (ONE) 273
1 Fading Hope (MID) 51
1 Hard Evidence (MH2) 46
1 Essence Scatter (M19) 54
1 Negate (RIX) 44
1 Heartless Act (IKO) 91
1 Ranger Class (AFR) 202
1 Courier's Briefcase (SNC) 142
1 Paradise Druid (WAR) 171
1 Inscription of Abundance (ZNR) 186
1 Gala Greeters (SNC) 148
1 Coldsteel Heart (CSP) 136
1 Aether Channeler (DMU) 42
1 Midnight Clock (ELD) 54
1 Chatterfang, Squirrel General (MH2) 151
1 Chitterspitter (MH2) 153
1 Tireless Tracker (SOI) 233
1 Whirler Rogue (ORI) 83
1 Binding the Old Gods (KHM) 206
1 Ertai Resurrected (DMU) 199
1 Shark Typhoon (IKO) 67
1 Junk Winder (MH2) 48
1 Titan of Industry (SNC) 159
1 Koma, Cosmos Serpent (KHM) 221
1 Cityscape Leveler (BRO) 233"""

In [ ]:
# TODO: Load deck from SealedDeck.tech
# Consider Card ordering.
MY_DECK = Deck.from_string(mine, "Mine")
THEIR_DECK = Deck.from_string(thiers, "Thiers")
print_deck_comparison(MY_DECK, THEIR_DECK)

In [ ]:
card_pool = """1 Deserted Beach (MID) 260
1 Glacial Fortress (XLN) 255
1 Drowned Catacomb (XLN) 253
1 Shipwreck Marsh (MID) 267
1 Dragonskull Summit (XLN) 252
1 Haunted Ridge (MID) 263
1 Rockfall Vale (MID) 266
1 Rootbound Crag (XLN) 256
1 Overgrown Farmland (MID) 265
1 Sunpetal Grove (XLN) 257
1 Isolated Chapel (DOM) 241
1 Shattered Sanctum (VOW) 264
1 Stormcarved Coast (VOW) 265
1 Sulfur Falls (DOM) 247
1 Deathcap Glade (VOW) 261
1 Woodland Cemetery (DOM) 248
1 Clifftop Retreat (DOM) 239
1 Sundown Pass (VOW) 266
1 Dreamroot Cascade (VOW) 262
1 Hinterland Harbor (DOM) 240
1 Savai Triome (IKO) 253
1 Ketria Triome (IKO) 250
1 Indatha Triome (IKO) 248
1 Raugrin Triome (IKO) 251
1 Zagoth Triome (IKO) 259
1 Fabled Passage (ELD) 244
1 Defiant Strike (WAR) 9
1 Esper Sentinel (MH2) 12
1 Favored Hoplite (THS) 13
1 Gods Willing (M20) 19
1 Homestead Courage (MID) 24
1 Fading Hope (MID) 51
1 Hard Evidence (MH2) 46
1 Slip Out the Back (SNC) 62
1 Cryptbreaker (EMN) 86
1 Stitcher's Supplier (M19) 121
1 Reckless Rage (RIX) 110
1 Ascendant Packleader (VOW) 186
1 Feat of Resistance (M21) 19
1 Illuminator Virtuoso (SNC) 17
1 Valorous Stance (VOW) 42
1 Ensoul Artifact (M15) 54
1 Essence Scatter (M19) 54
1 Negate (RIX) 44
1 Rise and Shine (MH2) 58
1 Stormchaser Drake (VOW) 82
1 Dreadhorde Invasion (WAR) 86
1 Heartless Act (IKO) 91
1 Mire Triton (THB) 105
1 Skyclave Shade (ZNR) 125
1 Cathartic Pyre (MID) 133
1 Dreadhorde Arcanist (WAR) 125
1 Obliterating Bolt (BRO) 145
1 Temur Battle Rage (FRF) 116
1 Courier's Briefcase (SNC) 142
1 Gala Greeters (SNC) 148
1 Inscription of Abundance (ZNR) 186
1 Paradise Druid (WAR) 171
1 Ranger Class (AFR) 202
1 Scavenging Ooze (M21) 204
1 Battlewise Hoplite (THS) 189
1 Bloodtithe Harvester (VOW) 232
1 Black Market Tycoon (SNC) 167
1 Conclave Mentor (M21) 216
1 Dromoka's Command (DTK) 221
1 Despark (WAR) 190
1 Humiliate (STX) 193
1 Priest of Fell Rites (MH2) 208
1 Expressive Iteration (STX) 186
1 Sprite Dragon (IKO) 211
1 Winding Constrictor (KLR) 216
1 Rip Apart (STX) 225
1 Tenth District Legionnaire (WAR) 222
1 Combine Chrysalis (MH2) 191
1 Coldsteel Heart (CSP) 136
1 Elite Spellbinder (STX) 17
1 Aether Channeler (DMU) 42
1 Midnight Clock (ELD) 54
1 Drana, Liberator of Malakir (JMP) 225
1 Murderous Rider (ELD) 97
1 Breya's Apprentice (MH2) 117
1 Flames of the Firebrand (JMP) 317
1 Krenko, Tin Street Kingpin (WAR) 137
1 Seasoned Pyromancer (MH1) 145
1 Chatterfang, Squirrel General (MH2) 151
1 Chitterspitter (MH2) 153
1 Rishkar, Peema Renegade (JMP) 425
1 Tireless Tracker (SOI) 233
1 Deputy of Detention (RNA) 165
1 Electrolyze (STA) 60
1 Feather, the Redeemed (WAR) 197
1 Nettlecyst (MH2) 231
1 Basri's Lieutenant (M21) 9
1 Cast Out (AKR) 9
1 Late to Dinner (MH2) 19
1 Memory Deluge (MID) 62
1 Whirler Rogue (ORI) 83
1 Blood for Bones (M20) 89
1 Big Score (SNC) 102
1 Jaya, Fiery Negotiator (DMU) 133
1 The Elder Dragon War (DMU) 121
1 Ulvenwald Oddity (VOW) 225
1 Vivien, Arkbow Ranger (M20) 199
1 Ertai Resurrected (DMU) 199
1 Binding the Old Gods (KHM) 206
1 Solemn Simulacrum (M21) 239
1 Angel of Sanctions (AKR) 1
1 Archfiend of Ifnir (AKR) 91
1 Liliana's Mastery (AKR) 113
1 The Cruelty of Gix (DMU) 87
1 Ox of Agonas (THB) 147
1 Verdurous Gearhulk (KLR) 185
1 Enter the God-Eternals (WAR) 196
1 Escape to the Wilds (ELD) 189
1 Wavesifter (MH2) 217
1 Sanctuary Warden (SNC) 30
1 Shark Typhoon (IKO) 67
1 Taste of Death (ELD) 320
1 Olivia, Crimson Bride (VOW) 245
1 Junk Winder (MH2) 48
1 Drakuseth, Maw of Flames (M20) 136
1 Titan of Industry (SNC) 159
1 Koma, Cosmos Serpent (KHM) 221
1 Platinum Angel (M10) 218
1 Cityscape Leveler (BRO) 233
1 Dark Salvation (J21) 308
"""
CARD_POOL = Deck.from_string(card_pool, "Pool")
CARD_POOL.maindeck

In [ ]:
trick_list = list()
for x in CARD_POOL.maindeck:
    if ("target" in x.ORACLE or "creature" in x.ORACLE or "prevent" in x.ORACLE) and ("Instant" in x.TYPES or "Flash" in x.ORACLE):
        trick_list.append(x)

def sort_key(x): return (WUBRG_COLOR_INDEXES[x.CAST_IDENTITY], x.CMC, RARITY_INDEXES[x.RARITY], x.NAME)
trick_list = sorted(trick_list, key=sort_key)   

for x in trick_list:
    print(f'"{x.NAME}","{x.MANA_COST}","{x.TYPE_LINE}","{x.RARITY}","{x.URL}"')

In [ ]:
land_list = list()
for x in CARD_POOL.maindeck:
    if "Land" in x.TYPES:
        land_list.append(x)
land_list

In [ ]:
def sort_key(x): return (x.CMC, WUBRG_COLOR_INDEXES[x.CAST_IDENTITY], RARITY_INDEXES[x.RARITY], x.NAME)
cards = sorted(MY_DECK.maindeck, key=sort_key)

for x in MY_DECK.maindeck:
    if "target" in x.ORACLE:
        print(f'"{x.NAME}","{x.MANA_COST}","{x.TYPE_LINE}","{x.RARITY}","{x.URL}"')

In [ ]:
deck = LimitedDeck.from_id("b20a97b818f3418b94a8f4e7584398a8")

In [ ]:
deck.draft

# 17 Lands Requester Testing

In [ ]:
import sys
sys.path.append('..')
from core.utilities.notebook_setups.notebook_display import *
set_up(log_lvl=LogLvl.DEBUG)
from core.data_requesting import Request17Lands
requester = Request17Lands(2, 5, 0.5)

In [ ]:
card_evals = requester.get_card_evaluations('ONE')
card_evals

In [ ]:
print(f"Length 'cards': {len(card_evals['cards'])}")
print(f"Length 'dates': {len(card_evals['dates'])}")
print(f"Length 'data': {len(card_evals['data'])}")
print(f"Length 'sub_data': {len(card_evals['data'][0])}")
len(card_evals['cards']) / 265

In [ ]:
for day in card_evals['data']

In [ ]:
set(card_evals['cards'][0:30])

In [ ]:
for x in range(0, len(card_evals['cards']), 30):
    print(card_evals['cards'][x])

In [ ]:
from core.game_metadata import SetMetadata, FormatMetadata, Card
data = SetMetadata.get_metadata('NEO')
data

In [ ]:
data.CARD_LIST

In [ ]:
data = pd.read_csv(r'C:\Users\Zachary\Downloads\game_data_public.ONE.PremierDraft\game_data_public.ONE.PremierDraft.csv')

In [ ]:
data

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
for x in data.columns:
    skip = False
    skip_list = ['opening_hand_', 'drawn_', 'tutored_', 'deck_', 'sideboard_']
    for y in skip_list:
        if x.startswith(y):
            skip = True
            break
            
    if skip:
        continue
    print(x)

In [ ]:
data[['draft_id', 'match_number', 'num_mulligans', 'won', 'user_n_games_bucket', 'user_game_win_rate_bucket']][data['num_mulligans'] >= 1]

# Alchemy Scratch Work

In [ ]:
import sys
sys.path.append('..')
from core.utilities.notebook_setups.notebook_display import *
set_up(log_lvl=LogLvl.DEBUG)

from core.game_metadata import CardManager
CardManager.load_cache_from_file()

from core.data_fetching import SetManager

KEY_COLS = ['# Seen', 'ALSA', '# Picked', 'ATA', 'GP WR', 'GP GW', 'OH WR', 'GD WR', 'GIH WR', 'IWD', 
            'Rarity', 'Color', 'Cast Color', 'CMC', 'Type Line', 'Tier', 'Rank']

set_data = SetManager('Y23BRO', load_history=False)
set_data.check_for_updates()

In [ ]:
alchemy_cards = [x for x in set_data.CARDS if x.IS_DIGITAL]
alchemy_cards

In [ ]:
summary_data = set_data.BO1.card_frame(deck_color='', summary=True).sort_values('Percentile', ascending=False)
summary_data[KEY_COLS]

In [ ]:
alchemy_data = set_data.BO1.card_frame(name=alchemy_cards, deck_color='', summary=True).sort_values('Percentile', ascending=False)
alchemy_data[KEY_COLS]

# CSV Scratch

In [ ]:
import numpy as np
import pandas as pd

scryfall = pd.read_csv(r'C:\Users\Zachary\Downloads\search.csv')
scryfall = scryfall.drop(['multiverse_id', 'mtgo_id', 'lang', 'artist', 'usd_price', 'usd_foil_price', 'eur_price', 'tix_price', 'image_uri', 'scryfall_uri'], axis=1)
lands_17 = pd.read_csv(r'C:\Users\Zachary\Downloads\cards.csv')

In [ ]:
scryfall

In [ ]:
lands_17

In [ ]:
set(scryfall['name']) ^ set(lands_17['Card Name'])

# Review (Vexisto)

In [ ]:
import sys
sys.path.append('..')
from core.utilities.notebook_setups.notebook_display import *
set_up(log_lvl=LogLvl.DEBUG)

In [ ]:
from time import sleep
from core.game_metadata.game_objects.Card import Card, CardManager

In [ ]:
SET = 'MOM'
DATA_ROOT =  f'C:\\Users\\Zachary\\Coding\\GitHub\\17LandsData\\{SET}\\Tiers\\'
pd.set_option('display.max_rows', 400)
CardManager.load_cache_from_file()
# TODO: Load card cache to not qury scryfall for every card.

In [ ]:
def patch_chord_excel(frame):
    # Patch
    # Fixing card names scryfall cannot parse.
    misnamed = {
        'Invastion of Xerex': "Invasion of Xerex",
        'Invastion of Amonkhet': "Invasion of Amonkhet",
        'Invasion of Ragatha': "Invasion of Regatha",
        'Ariel Boost': "Aerial Boost",
        'Sun-Blessed Gaurdian': "Sun-Blessed Guardian",
        'Etali, Primal Conquerer': "Etali, Primal Conqueror",
        'Radah, Colition Warlord': "Radha, Coalition Warlord",
        'Seizan the Pervert': "Seizan, Perverter of Truths",
        "Horobi, Death's Whisper": "Horobi, Death's Wail",
        "Tetsuko Umezawa": "Tetsuko Umezawa, Fugitive"
    }
    frame = frame.replace({"Card Name": misnamed})

    # Patch
    # Converting tap land cycle grade into per-card grade.
    # 38 	Tap Land Cycle 	C+ 	C+
    frame = frame.drop([38], axis=0)
    extra = {
        "Card Name": 
        ['Bloodfell Caves',
         'Blossoming Sands',
         'Dismal Backwater',
         'Jungle Hollow',
         'Rugged Highlands',
         'Scoured Barrens',
         'Swiftwater Cliffs',
         'Thornwood Falls',
         'Tranquil Cove',
         'Wind-Scarred Crag'],
        "Marc": ["C+"] * 10,
        "Alex": ["C+"] * 10,
    }
    xtra = pd.DataFrame(extra)
    frame = pd.concat([frame, xtra])
    
    # Patch
    # Adding missing cards.
    missing = {
        "Card Name": [
            "Keruga, the Macrosage",
            "Ezuri, Claw of Progress",
            "Captain Lannery Storm",
        ],
        "Marc": [
            "SB",
            "SB",
            "SB",
        ],
        "Alex": [
            "SB",
            "SB",
            "SB",
        ],
    }
    xtra = pd.DataFrame(missing)
    frame = pd.concat([frame, xtra])
    
    name_patch = {card_name: CardManager.from_name(card_name) for card_name in frame["Card Name"]}
    frame = frame.replace({"Card Name": name_patch})
    
    return frame

In [ ]:
def parse_chord_excel(file_name, patch_func=None):
    frame = pd.read_excel(DATA_ROOT + file_name)
    frame = frame.dropna(axis=0, how='all')
    frame = frame.drop(["COLOR", "RARITY"], axis=1)
    frame.columns = ['Card Name', 'Marc', 'Alex']
    frame = frame[['Card Name', 'Alex', 'Marc']]
    if patch_func is not None:
        frame = patch_func(frame)
    return frame

In [ ]:
# https://docs.google.com/spreadsheets/d/1KDTva5x_PmzAXPXtas4yrxZgU6TblUqAMuZfZBdnrus/edit#gid=0
# https://docs.google.com/spreadsheets/d/1KDTva5x_PmzAXPXtas4yrxZgU6TblUqAMuZfZBdnrus/export?format=xlsx
set_grades = parse_chord_excel('MOM Gradings.xlsx', patch_chord_excel)
set_grades = set_grades.set_index('Card Name')
# The index is made of 'Card' objects. Convert to strings for better interoperability.
set_grades.index = [card.FULL_NAME for card in set_grades.index]
set_grades

In [ ]:
import requests
from docx import Document
from docx.shared import Pt, RGBColor, Inches
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from io import BytesIO
from PIL import Image
import docx.opc.constants
import docx.oxml.ns

In [ ]:
card_order = [
    "Rampaging Geoderm",
    "Elvish Vatkeeper",
    "Sculpted Perfection",
    "Mutagen Connoisseur",
    "Joyful Stormsculptor",
    "Mirror-Shield Hoplite",
    "Botanical Brawler",
    "Halo Forager",
    "Stormclaw Rager",
    "Marshal of Zhalfir",
    "Invasion of Xerex // Vertex Paladin",
    "Invasion of Ergamon // Truga Cliffcharger",
    "Invasion of Amonkhet // Lazotep Convert",
    "Invasion of Azgol // Ashen Reaper",
    "Invasion of Moag // Bloomwielder Dryads",
    "Invasion of Kaladesh // Aetherwing, Golden-Scale Flagship",
    "Invasion of New Capenna // Holy Frazzle-Cannon",
    "Invasion of Pyrulea // Gargantuan Slabhorn",
    "Invasion of Lorwyn // Winnowing Forces",
    "Invasion of Kylem // Valor's Reach Tag Team",
    "Invasion of Dominaria // Serra Faithkeeper",
    "Invasion of Belenon // Belenon War Anthem",
    "Invasion of Kamigawa // Rooftop Saboteurs",
    "Invasion of Vryn // Overloaded Mage-Ring",
    "Invasion of Eldraine // Prickle Faeries",
    "Invasion of Ulgrotha // Grandmother Ravi Sengir",
    "Invasion of Mercadia // Kyren Flamewright",
    "Invasion of Regatha // Disciples of the Inferno",
    "Invasion of Muraganda // Primordial Plasm",
    "Invasion of Zendikar // Awakened Skyclave",
    "Urn of Godfire",
    "Flywheel Racer",
    "Kitesail",
    "Halo Hopper",
    "Skittering Surveyor ",
    "Phyrexian Archivist ",
    "Tranquil Cove",
    "Enduring Bondwarden",
    "Kor Halberd",
    "Tarkir Duneshaper",
    "Aerial Boost",
    "Alabaster Host Sanctifier",
    "Angelic Intervention",
    "Realmbreaker's Grasp",
    "Sunder the Gateway",
    "Swordsworn Cavalier",
    "Attentive Skywarden",
    "Cut Short",
    "Kithkin Billyrider",
    "Scrollshift",
    "Sigiled Sentinel",
    "Bola Slinger",
    "Inspired Charge",
    "Knight of the New Coalition",
    "Golden-Scale Aeronaut",
    "Infected Defector",
    "Alabaster Host Intercessor ",
    "Elspeth's Smite",
    "Surge of Salvation",
    "Norn's Inquisitor",
    "Sun-Blessed Guardian // Furnace-Blessed Conqueror",
    "Phyrexian Awakening",
    "Phyrexian Censor",
    "Seal From Existence",
    "Seraph of New Capenna",
    "Zhalfirin Lancer",
    "Tiller of Flesh",
    "Assimilate Essence",
    "Disturbing Conversion",
    "Expedition Lookout",
    "Moment of Truth",
    "Negate",
    "Order of the Mirror",
    "Saiba Cryptomancer",
    "Stasis Field",
    "Zhalfirin Shapecraft",
    "Ephara's Dispersal",
    "Eyes of Gitaxias",
    "Furtive Analyst",
    "Preening Champion",
    "Meeting of Minds",
    "Oculus Whelp",
    "Protocol Knight",
    "Temporal Cleansing",
    "Halo-Charged Skaab",
    "Thunderhead Squadron",
    "Tidal Terror",
    "Captive Weird",
    "Omen Hawker",
    "Change the Equation ",
    "Oracle of Tragedy",
    "Skyclave Aerialist",
    "Xerex Strobe-Knight",
    "Wicked Slumber",
    "Astral Wingspan",
    "Corruption of Towashi",
    "Artistic Refusal",
    "Corrupted Conviction",
    "Ichor Drinker",
    "Mirrodin Avenged",
    "Aetherblade Agent",
    "Bladed Battle-Fan",
    "Dreg Recycler",
    "Final Flourish",
    "Nezumi Informant ",
    "Traumatic Revelation",
    "Etched Familiar",
    "Flitting Guerrilla",
    "Ichor Shade",
    "Unseal the Necropolis",
    "Vanquish the Weak",
    "Consuming Aetherborn",
    "Deadly Derision",
    "Tenured Oilcaster",
    "Etched Host Doombringer",
    "Failed Conversion",
    "Gloomfang Mauler",
    "Scorn-Blade Berserker",
    "Blightreaper Thallid",
    "Gift of Compleation",
    "Collective Nightmare",
    "Compleated Huntmaster",
    "Glistening Deluge",
    "Render Inert",
    "Nezumi Freewheeler",
    "Merciless Repurposing",
    "Phyrexian Gargantua",
    "Akki Scrapchomper",
    "Beamtown Beatstick",
    "Coming in Hot",
    "Mirran Banesplitter",
    "Burning Sun's Fury",
    "Pyretic Prankster",
    "Ral's Reinforcements",
    "Thrashing Frontliner",
    "Trailblazing Historian",
    "Volcanic Spite",
    "Wrenn's Resolve",
    "Hangar Scrounger",
    "Karsus Depthguard",
    "Marauding Dreadship",
    "Searing Barb",
    "Redcap Heelslasher",
    "War-Trained Slasher",
    "Onakke Javelineer",
    "Furnace Host Charger",
    "Shatter The Source",
    "Lithomantic Barrage",
    "Furnace Gremlin",
    "Khenra Spellspear",
    "Furnace Reins",
    "Harried Artisan",
    "Scrappy Bruiser",
    "Stoke the Flames",
    "Fearless Skald",
    "Ramosian Greatsword",
    "Shivan Branch-Burner",
    "Arachnoid Adaptation",
    "Placid Rottentail",
    "Seed of Hope",
    "Atraxa's Fall",
    "Cosmic Hunger",
    "Iridescent Blademaster ",
    "Portent Tracker",
    "Vengeant Earth",
    "Wary Thespian",
    "Blighted Burgeoning",
    "Overgrown Pest",
    "Serpent-Blade Assailant",
    "War HIstorian",
    "Chomping Kavu",
    "Converter Beast",
    "Crystal Carapace",
    "Fertilid's Favor",
    "Bonded Herdbeast",
    "Wild Escort",
    "Timberland Ancient",
    "Herbology Instructor",
    "Streetwise Negotiator",
    "Kami of Whispered Hopes",
    "Sandstalker Moloch",
    "Tandem Takedown",
    "Gnottvold Hermit",
    "Storm the Seedcore",
    "Ravenous Sailback",
    "Tangled Skyline",
    "Copper Host Crusher",
]

card_order = [CardManager.from_name(card_name).FULL_NAME for card_name in card_order]

test_order = [
    "Halo Forager",
    "Invasion of Amonkhet // Lazotep Convert",
    "Aetherblade Agent",
]

test_order = [CardManager.from_name(card_name).FULL_NAME for card_name in test_order]

In [ ]:
from functools import cache

@cache
def get_card_data(card_name):
    search_url = f"https://api.scryfall.com/cards/search?q={card_name}"
    response = requests.get(search_url)
    data = response.json()
    sleep(0.1)

    if data["total_cards"] > 0:
        card_data = data["data"][0]
        return card_data
    else:
        return None
    
@cache
def get_image_data(card_image_url):
    response = requests.get(card_image_url)
    image_data = response.content
    sleep(0.1)
    return image_data

In [ ]:
FONT_NAME = 'Calibri Light'
FONT_SIZE = Pt(16)
FONT_COLOR = RGBColor(0x1F, 0x37, 0x63)

def apply_default_font(font):
    font.name = FONT_NAME
    font.size = FONT_SIZE
    font.color.rgb = FONT_COLOR

def add_hyperlink(document, run, url):
    r_id = document.part.relate_to(url, docx.opc.constants.RELATIONSHIP_TYPE.HYPERLINK, is_external=True)
    
    r = run._r
    r_hyperlink = docx.oxml.shared.OxmlElement('w:hyperlink')
    r_hyperlink.set(docx.oxml.shared.qn('r:id'), r_id)
    r_hyperlink.set(docx.oxml.shared.qn('w:history'), '1')

    new_run = docx.oxml.shared.OxmlElement('w:r')
    r_properties = docx.oxml.shared.OxmlElement('w:rPr')

    new_run.append(r_properties)
    new_run.text = run.text
    r_hyperlink.append(new_run)

    run._r.getparent().replace(run._r, r_hyperlink)
    apply_default_font(run.font)

    return run

# Temporary file to save images to, before adding into docx.
TEMP_LOC = 'C:\\Users\\Zachary\\Coding\\GitHub\\DraftMetaReporter\\Notebooks\\temp.jpeg'
TEMP_FRONT_LOC = 'C:\\Users\\Zachary\\Coding\\GitHub\\DraftMetaReporter\\Notebooks\\f.jpeg'
TEMP_BACK_LOC = 'C:\\Users\\Zachary\\Coding\\GitHub\\DraftMetaReporter\\Notebooks\\b.jpeg'
SCALE = 1
HEIGHT = 3.24 * SCALE
WIDTH = 2.29 * SCALE

def format_image(card_image_url, rotate, out_loc):
    image_data = get_image_data(card_image_url)

    with BytesIO(image_data) as b_io:
        with Image.open(b_io) as img:
            new_height = HEIGHT  # 3.24 inches
            new_width = WIDTH    # 2.29 inches

            if rotate:
                img = img.rotate(270, expand=True)              # Rotate image by -90 degrees
                new_height, new_width = new_width, new_height   # Switch the height and width, since we've rotated.

            # Save the image fetched from the web as a JPEG, then use that file location to add the image into the docx.
            img.save(out_loc, format='JPEG')
    return new_height, new_width

def add_images(document, card_data):
    height = 0
    width = 0
    
    if "layout" in card_data and card_data["layout"] == "transform":
        front_face = card_data["card_faces"][0]
        rotate = front_face['type_line'] == 'Battle — Siege'
        h, w = format_image(front_face["image_uris"]["border_crop"], rotate, TEMP_FRONT_LOC)
        height = max(height, h)
        width += w
        front = Image.open(TEMP_FRONT_LOC)
        
        back_face = card_data["card_faces"][1]
        h, w = format_image(back_face["image_uris"]["border_crop"], False, TEMP_BACK_LOC)
        height = max(height, h)
        width += w
        back = Image.open(TEMP_BACK_LOC)
       
        new = Image.new('RGBA', (front.size[0] + back.size[0], max(front.size[1], back.size[1])), (0, 0, 0, 15))
        new.paste(front, (0, (new.size[1] - front.size[1]) // 2))
        new.paste(back, (front.size[0], (new.size[1] - back.size[1]) // 2))
        new.save(TEMP_LOC, format='PNG')
    else:
        h, w = format_image(card_data["image_uris"]["border_crop"], False, TEMP_LOC)
        height = h
        width = w
        
    paragraph = document.add_paragraph()
    picture = paragraph.add_run().add_picture(TEMP_LOC, height=Inches(height), width=Inches(width))
    picture.vertical_alignment = 3
    # picture.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    picture.inline_format = True
    
def add_images(cell, card_data):
    height = 0
    width = 0
    
    if "layout" in card_data and card_data["layout"] == "transform":
        front_face = card_data["card_faces"][0]
        rotate = front_face['type_line'] == 'Battle — Siege'
        h, w = format_image(front_face["image_uris"]["border_crop"], rotate, TEMP_FRONT_LOC)
        height = max(height, h)
        width += w
        front = Image.open(TEMP_FRONT_LOC)
        
        back_face = card_data["card_faces"][1]
        h, w = format_image(back_face["image_uris"]["border_crop"], False, TEMP_BACK_LOC)
        height = max(height, h)
        width += w
        back = Image.open(TEMP_BACK_LOC)
       
        new = Image.new('RGBA', (front.size[0] + back.size[0], max(front.size[1], back.size[1])), (0, 0, 0, 15))
        new.paste(front, (0, (new.size[1] - front.size[1]) // 2))
        new.paste(back, (front.size[0], (new.size[1] - back.size[1]) // 2))
        new.save(TEMP_LOC, format='PNG')
    else:
        h, w = format_image(card_data["image_uris"]["border_crop"], False, TEMP_LOC)
        height = h
        width = w
        
    paragraph = paragraph = cell.paragraphs[0]
    paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    picture = paragraph.add_run().add_picture(TEMP_LOC, height=Inches(height), width=Inches(width))
    
def add_grade(cell, card_name, grader):
    grade = set_grades.loc[card_name][grader].strip()
    if grade.startswith('BA'):
        grade = f"{grade[2:]}, Build-Around"
    if grade.startswith('SYN'):
        grade = f"{grade[3:]}, Synergy"
    
    cell.text = f"{grader}: {grade}"
    paragraph = cell.paragraphs[0]
    paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    run = paragraph.runs[0]
    run.bold = True
    font = run.font
    font.name = FONT_NAME
    font.size = Pt(12)
    font.color.rgb = RGBColor(0x00, 0x00, 0x00)

In [ ]:
def add_card_to_document(document, card_name, card_data):
    style = document.styles['Heading 3']
    apply_default_font(style.font)

    heading = document.add_heading("", level=3)
    heading.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    run = heading.add_run(card_name)
    card_url = card_data["scryfall_uri"]
    
    add_hyperlink(document, run, card_url)
    table = document.add_table(rows=3, cols=2)
    add_images(table.cell(0, 0), card_data)
    table.cell(0, 0).merge(table.cell(0, 1))
    
    add_grade(table.cell(1, 0), card_name, 'Marc')
    add_grade(table.cell(1, 1), card_name, 'Alex')
    
    paragraph = table.cell(2, 0).paragraphs[0]
    paragraph.style = 'List Bullet'
    table.cell(2, 0).merge(table.cell(2, 1))
    document.add_page_break()

def create_document(card_names, doc_name="test.docx"):
    document = Document()

    for card_name in card_names:
        card_name = card_name.strip()
        card_data = get_card_data(card_name)

        if card_data:
            add_card_to_document(document, card_name, card_data)
        else:
            print(f"Card not found: {card_name}")

    document.save(doc_name)

In [ ]:
create_document(test_order, "test.docx")

In [ ]:
raise Exception("Oops! Clicked too fast")

In [ ]:
create_document(card_order, "Template.docx")